# Data Processing

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
np.seterr(divide = 'ignore') 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#datasets loading
data_train_X = pd.read_csv('/content/drive/MyDrive/Data Science Pro/Projekt końcowy/DATA before processing/input_data_train_X')
data_train_y = pd.read_csv('/content/drive/MyDrive/Data Science Pro/Projekt końcowy/DATA before processing/input_data_train_y')
data_test_X = pd.read_csv('/content/drive/MyDrive/Data Science Pro/Projekt końcowy/DATA before processing/input_data_test_X')
data_test_y = pd.read_csv('/content/drive/MyDrive/Data Science Pro/Projekt końcowy/DATA before processing/input_data_test_y')
working_site_data = pd.read_csv('/content/drive/MyDrive/Data Science Pro/Projekt końcowy/Original DATA/working_site_metadata.csv')

In [ ]:
data_train_X.shape

## This is fourth version of dataset processing (second version of final dataset). 

---

In the fourth approach <b>maximum</b> and <b>std</b> were computed from time-series features on 8-hour time periods with 4 hour increments. 

Only the features concerning max and counts values were used; sums and averages were left out.

From extra dataset <b>only mining_hazard_assessment</b> was used.

In [ ]:
def ts_col_names(first_ts_col, last_ts_col):
  return list(data_train_X.iloc[:, first_ts_col: last_ts_col].columns)

In [ ]:
#count_eX
counts_e2 = ts_col_names(13, 37)
counts_e3 = ts_col_names(37, 61)
counts_e4 = ts_col_names(61, 85)
counts_e5 = ts_col_names(85, 109)
counts_e6plus = ts_col_names(109, 133)
#sum_eX
sum_e2 = ts_col_names(133, 157)
sum_e3 = ts_col_names(157, 181)
sum_e4 = ts_col_names(181, 205)
sum_e5 = ts_col_names(205, 229)
sum_e6plus = ts_col_names(229, 253)
#total_number_of_bumps
total_number_of_bumps = ts_col_names(253, 277)
#number_of_rock_bursts
num_rock_bursts = ts_col_names(277,301)
#number_of_destressing_blasts
num_des_blasts = ts_col_names(301, 325)
#highest_bump_energy
h_bump_energy = ts_col_names(325, 349)
#other time series
max_gactivities = ts_col_names(349, 373)
max_genergies = ts_col_names(373, 397)
avg_gactivities = ts_col_names(397, 421)
avg_genergies = ts_col_names(421, 445)
max_diff_gactivities = ts_col_names(445, 469)
max_diff_genergies = ts_col_names(469, 493)
avg_diff_gactivities = ts_col_names(493, 517)
avg_diff_genergies = ts_col_names(517, 541)

Time series aggregation

In [ ]:
def features_extracting(df,ts, time_period, increment):

  slide = 0
  init_col_name = ts[0][:-2]
  new_max_cols, new_std_cols = [], []

  while (slide+time_period) <= len(ts):

    #max
    col_name = f'{slide}-{time_period+slide}h_max_' + init_col_name
    df[col_name] = df[ts[slide: time_period+slide]].max(axis=1)
    new_max_cols.append(col_name)

    #std
    col_name = f'{slide}-{time_period+slide}h_std_' + init_col_name
    df[col_name] = df[ts[slide: time_period+slide]].std(axis=1)
    new_std_cols.append(col_name)

    slide += increment

  return df, new_max_cols, new_std_cols

In [ ]:
ts_to_agg = [
    counts_e2, counts_e3, 
    counts_e4, counts_e5, counts_e6plus,
    num_rock_bursts, num_des_blasts, 
    h_bump_energy, max_gactivities,
     max_genergies, max_diff_gactivities, 
     max_diff_genergies
     ]

In [ ]:
max_cols, std_cols = [], []

for ts in ts_to_agg:
  data_train_X, new_max_cols, new_std_cols = features_extracting(data_train_X, ts, 8, 4)
  features_extracting(data_test_X, ts, 8, 4)

  max_cols.append(new_max_cols)
  std_cols.append(new_std_cols)

Mining_hazard_assessment feature appending from additional dataset

In [ ]:
def find_h_assessment(x, dictionary):
    for key in dictionary:
        if x == key:
            return (dictionary[key][1])

In [ ]:
dict_id = {}
for i, row in working_site_data.iterrows():
    dict_id[row['main_working_id']] = (row['main_working_height'], row['mining_hazard_assessment'])

In [ ]:
data_train_X['mining_hazard_assessment'] = data_train_X['main_working_id'].apply(lambda x: find_h_assessment(x, dict_id))
data_test_X['mining_hazard_assessment'] = data_test_X['main_working_id'].apply(lambda x: find_h_assessment(x, dict_id))

# Categorical variables

latest_seismoacoustic_assessment 

In [ ]:
data_train_X['latest_seismoacoustic_assessment'].value_counts()

In [ ]:
data_train_X['latest_seismoacoustic_assessment'].replace('d', 'c', inplace=True)
#'d' category contains only 48 instances. What's even more important, in the test data there is no category 'd'.
#'d' category will be replaced to 'c'.

In [ ]:
data_train_X = pd.get_dummies(data_train_X, drop_first=True)

In [ ]:
data_test_X = pd.get_dummies(data_test_X, drop_first=True)

In [ ]:
data_test_X = data_test_X.reindex(columns = data_train_X.columns, fill_value=0)

# Removing features

Original time series features that were aggregated now will be dropped from dataset.

In [ ]:
for ts in ts_to_agg:
  data_train_X.drop(ts, axis=1, inplace=True)
  data_test_X.drop(ts, axis=1, inplace=True)

In [ ]:
#main_working_id will be dropped
data_train_X.drop('main_working_id', axis=1, inplace=True)
data_test_X.drop('main_working_id', axis=1, inplace=True)

In [ ]:
#also total_number_of bumps will be dropped, as it's exactly the same information as summed
#counts of bumps(count_e*), they're just summed values from all detectors 
data_train_X.drop(total_number_of_bumps, axis=1, inplace=True)
data_test_X.drop(total_number_of_bumps, axis=1, inplace=True)

# Dataset overview

In [ ]:
data_train_X.shape

In [ ]:
data_test_X.shape

In [ ]:
import itertools

In [ ]:
flatten_max_cols = list(itertools.chain.from_iterable(max_cols))

In [ ]:
nrows = len(flatten_max_cols)//3+1
fig, axs = plt.subplots(nrows=nrows, ncols=3, figsize=(15, 50))

for i, column in enumerate(flatten_max_cols):

  ax = axs[i//3][i%3]
  ax.hist(data_train_X[column], alpha=0.5)
  ax.set_title(column)

plt.tight_layout()
plt.show()

In [ ]:
#storing variables
data_train_X.to_csv('/content/drive/MyDrive/Data Science Pro/Projekt końcowy/EXTRACTED DATA/X_train4', index=False)
data_test_X.to_csv('/content/drive/MyDrive/Data Science Pro/Projekt końcowy/EXTRACTED DATA/X_test4', index=False)
